# 所需套件

In [1]:
import jieba
import jieba.posseg as pseg
import jieba.analyse

import speech_recognition as sr
from pprint import pprint
from os import path

import requests
from bs4 import BeautifulSoup

import requests as req
import os
from xml.etree import ElementTree
import pyaudio
import wave
import sys

## 辨識使用者說什麼，轉成文字

In [2]:
def listen_user_says(my_key):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print ('Say Something!')
        audio = r.listen(source)
        print ('Done!')
    try:
        print("Microsoft Bing Voice Recognition thinks you said: ")
        # pprint(r.recognize_bing(audio, key=my_key, language="zh-CN", show_all=True))
        return r.recognize_bing(audio, key=my_key, language="zh-TW") 
    except sr.UnknownValueError:
        print("Microsoft Bing Voice Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Microsoft Bing Voice Recognition service; {0}".format(e))

## 傳入使用者所說的文字，進行分詞

In [3]:
def split_by_Jieba(text):
    jieba.set_dictionary("dict/dict.txt.big.txt")
    jieba.load_userdict("dict/userdict.txt")
    
    simple_list = []
    simple_list = list(jieba.cut(text))
    return simple_list

## 針對主題進行爬蟲

In [4]:
def scrape_by_theme(theme):
    base_url = "https://udn.com"
    start_url = base_url + "/news/index"
    req = requests.get(start_url)
    soupObj = BeautifulSoup(req.text, 'html.parser')
    sports=""

    print("in scraper, the theme is : " + theme)
    for link in soupObj.find_all('a'):
        s = set(link.contents)
        if theme in s:
            temp = link.get('href')
            sports = temp
            break

    sports_url = base_url + sports
    
    print(sports_url)
    req2 = requests.get(sports_url)
    print(req2.status_code)
    soupObj2 = BeautifulSoup(req2.text, 'html.parser')

    ob = soupObj2.find_all("div", attrs={'class': 'area'})
    top5_news = []

    for i in ob:
        x = i.find_next("h3")
        print(x.contents)
        if u"最新文章" in x.contents:
            top_news = x.next_sibling
            for item in top_news.find_all("h2"):
                top5_news.append(item.get_text())
            break
    return top5_news 

In [5]:
def toSound(news_list,api_key):

    headers = {'Ocp-Apim-Subscription-Key': api_key}
    get_token = req.session()
    access_token = get_token.post('https://api.cognitive.microsoft.com/sts/v1.0/issueToken', headers=headers).text

    headers = {"Content-type": "application/ssml+xml",
            "X-Microsoft-OutputFormat": "riff-16khz-16bit-mono-pcm",
            "Authorization": "Bearer " + access_token}

    # Changing voice output via SSML
    body = ElementTree.Element('speak', version='1.0')
    body.set('xml:lang', 'en-GB')
    voice = ElementTree.SubElement(body, 'voice')
    voice.set('xml:lang', 'en-GB')
    voice.set('xml:gender', 'Female')

    voice.set('name', 'Microsoft Server Speech Text to Speech Voice (zh-TW, Yating, Apollo)')

    # 建立資料夾
    get_record = req.session()
    os.makedirs('./output', exist_ok=True)
    

    i = 0
    for text in news_list:
        i+=1
        voice.text = text
        respond = get_record.post('https://speech.platform.bing.com/synthesize', data=ElementTree.tostring(body), headers=headers)
        
        audio_name = "news_"+ str(i)+ ".wav"
        with open('./output/' + audio_name, 'wb') as f:
            f.write(respond.content)

        # play the sound we just turn in

        CHUNK = 1024

        wf = wave.open("./output/"+audio_name, 'rb')

        p = pyaudio.PyAudio()

        stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                        channels=wf.getnchannels(),
                        rate=wf.getframerate(),
                        output=True)

        data = wf.readframes(CHUNK)

        while len(data) > 0:
            stream.write(data)
            data = wf.readframes(CHUNK)

        stream.stop_stream()
        stream.close()

        p.terminate()

# 來跑跑看吧～

In [8]:
# your key here
BING_KEY = ""

# 使用 bing speech api 音檔轉文字的服務
user_says = listen_user_says(BING_KEY)
print(user_says)

Say Something!
Done!
Microsoft Bing Voice Recognition thinks you said: 
我想知道運動的新聞


# jieba 分詞


In [7]:
sep_words = split_by_Jieba(user_says)
print(sep_words)

Building prefix dict from /Users/tp6han/Documents/python/scraper_bing_speech_api/dict/dict.txt.big.txt ...
Loading model from cache /var/folders/sl/zps8bzs557qgnvccrm1wtbx00000gn/T/jieba.u2386d3d8b4957de4a74d0f468bedc473.cache
Loading model cost 1.503 seconds.
Prefix dict has been built succesfully.


['我', '想', '知道', '全球', '的', '新聞']


In [9]:
tags = jieba.analyse.extract_tags(user_says, topK=1)
print(tags)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/sl/zps8bzs557qgnvccrm1wtbx00000gn/T/jieba.cache
Loading model cost 0.907 seconds.
Prefix dict has been built succesfully.


['運動']


In [36]:
print('產經')

產經


In [11]:
theme = str(tags[0])
print(theme)

運動


In [46]:
for item in sep_words:
    print("item >>> "+ item)
    if item == u'運動' or item == u'體育':
        theme = u'運動'
        print("user want sports news")
    elif item == u'股市':
        theme = u'股市'

NameError: name 'sep_words' is not defined

In [14]:
top5_news = scrape_by_theme(theme)
print(*top5_news, sep='\n')

in scraper, the theme is : 運動
https://udn.com/news/cate/2/7227
200
['運動頭條']
['最新文章', <a class="more" href="/news/breaknews/1/7#breaknews" target="_blank" title="more"></a>]
>>>>>   in main
田徑／相隔15年 吳佳如「撐」破全國
跆拳道／羅嘉翎「金」喜 世青賽連兩屆摘金
傷兵多、黑馬強 引出季後賽首輪三大話題
籃球／理查森替NBA 3x造勢 預測勇士衛冕成功
LPGA／樂天高球賽首輪 錢珮芸並列49名台將最佳


In [15]:
top5_news

['田徑／相隔15年 吳佳如「撐」破全國',
 '跆拳道／羅嘉翎「金」喜 世青賽連兩屆摘金',
 '傷兵多、黑馬強 引出季後賽首輪三大話題',
 '籃球／理查森替NBA 3x造勢 預測勇士衛冕成功',
 'LPGA／樂天高球賽首輪 錢珮芸並列49名台將最佳']

In [16]:
toSound(top5_news,BING_KEY)

### 運動 股市